In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_excel('/content/drive/MyDrive/NNFL-Assignment2/data55.xlsx',header=None)
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,0.3108,0.2933,0.2275,0.0994,0.1801,0.2200,0.2732,0.2862,0.2034,1
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,0.3085,0.3425,0.2990,0.1402,0.1235,0.1534,0.1901,0.2429,0.2120,1
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,0.2716,0.2374,0.1878,0.0983,0.0683,0.1503,0.1723,0.2339,0.1962,1
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,0.2898,0.2812,0.1578,0.0273,0.0673,0.1444,0.2070,0.2645,0.2828,1


In [5]:
datan = data.values
np.random.shuffle(datan)
X = datan[:,0:19]

m = X.shape[0] #no of examples
#X = (X- np.min(X, axis = 0))/(np.max(X, axis = 0)- np.min(X, axis = 0))
X = (X - X.mean(axis = 0))/(X.std(axis = 0))

y=datan[:,-1:] #output
y=np.where(y==0,-1,y)

In [6]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(149, 19) (17, 19) (42, 19) (149, 1) (17, 1) (42, 1)


**LINEAR**

In [7]:
X_size=int(X.size/19)
X_train_size=int(X_train.size/19)
delta=np.zeros(X_train_size)

In [8]:
kernel_matrix = np.zeros((X_size,X_size))
for i in range(X_size):
  for j in range(X_size):
    kernel_matrix[i][j]=np.dot(X[i].T,X[j])
kernel_matrix 

array([[15.62579914, -2.9945919 , 10.33561372, ..., 15.83616749,
         0.34396279,  8.44100835],
       [-2.9945919 , 10.94275471,  3.2236841 , ..., -3.49181784,
        -0.04360989,  0.51280425],
       [10.33561372,  3.2236841 , 14.05018957, ..., 10.14870618,
         0.72890778,  6.70299036],
       ...,
       [15.83616749, -3.49181784, 10.14870618, ..., 17.57486498,
         1.56167329,  8.58764213],
       [ 0.34396279, -0.04360989,  0.72890778, ...,  1.56167329,
         4.22074096, -0.81580317],
       [ 8.44100835,  0.51280425,  6.70299036, ...,  8.58764213,
        -0.81580317,  7.52006011]])

In [9]:
for i in range(X_train_size):
  hypothesis=0
  for j in range(X_train_size):
    hypothesis=hypothesis+(delta[j]*y_train[j]*kernel_matrix[j][i])
  if(hypothesis>=0):
    h=1
  else:
    h=-1
  if(h!=y_train[i]):
    delta[i]=delta[i]+1   

In [10]:
X_val_size=int(X_val.size/19)
count=0
for i in range(X_val_size):
  hypothesis=0
  for j in range(X_train_size):
    hypothesis=hypothesis+(delta[j]*y_train[j]*kernel_matrix[j][i+X_train_size])
  if(hypothesis>=0):
    h=1
  else:
    h=-1
  if(h==y_val[i]):
    count=count+1
accuracy=count/X_val_size
accuracy   




0.47058823529411764

In [25]:
##for testing data
X_test_size=int(X_test.size/19)
ypred=np.zeros(X_test_size)
for i in range(X_test_size):
  hypothesis=0
  for j in range(X_train_size):
    hypothesis=hypothesis+(delta[j]*y_train[j]*kernel_matrix[j][i+X_train_size+X_val_size])
  if(hypothesis>=0):
    ypred[i]=1
  else:
    ypred[i]=-1
TN = 0
TP = 0
FN = 0
FP = 0
for i in range(len(y_test)):
  if y_test[i]==-1:
    if ypred[i]==-1:
      TN += 1
    elif ypred[i]==1:
      FP += 1
  elif y_test[i]==1:
    if ypred[i]==-1:
      FN += 1
    elif ypred[i]==1:
      TP += 1
confusion_matrix = pd.DataFrame(data=[[TN,FP],[FN,TP]])
confusion_matrix


,0,1
0,19,2
1,0,21


In [26]:
SE = TP/(TP+FN)
SP = TN/(TN+FP)
Acc = (TP+TN)/(TP+TN+FP+FN)
print(SE)
print(SP)
print(Acc)

1.0
0.9047619047619048
0.9523809523809523


**POLYNOMIAL**

In [31]:
X_size=int(X.size/19)
X_train_size=int(X_train.size/19)
delta=np.zeros(X_train_size)
kernel_matrix=np.zeros((X_size,X_size))
for i in range(X_size):
  for j in range(X_size):
    kernel_matrix[i][j]=(np.dot(X[i].T,X[j])+1)**3
for i in range(X_train_size):
  hypothesis=0
  for j in range(X_train_size):
    hypothesis=hypothesis+(delta[j]*y_train[j]*kernel_matrix[j][i])
  if(hypothesis>=0):
    h=1
  else:
    h=-1
  if(h!=y_train[i]):
    delta[i]=delta[i]+1 

In [37]:
X_val_size=int(X_val.size/19)
count=0
for i in range(X_val_size):
  hypothesis=0
  for j in range(X_train_size):
    hypothesis=hypothesis+(delta[j]*y_train[j]*kernel_matrix[j][i+X_train_size])
  if(hypothesis>=0):
    h=1
  else:
    h=-1
  if(h==y_val[i]):
    count=count+1
accuracy=(count)/(X_val_size)
accuracy  

0.9090909090909091

In [43]:
##for testing data
X_test_size=int(X_test.size/19)
ypred=np.zeros(X_test_size)
for i in range(X_test_size):
  hypothesis=0
  for j in range(X_train_size):
    hypothesis=hypothesis+(delta[j]*y_train[j]*kernel_matrix[j][i+X_train_size+X_val_size])
  if(hypothesis>=0):
    ypred[i]=1
  else:
    ypred[i]=-1
TN = 0
TP = 0
FN = 0
FP = 0
for i in range(len(y_test)):
  if y_test[i]==-1:
    if ypred[i]==-1:
      TN += 1
    elif ypred[i]==1:
      FP += 1
  elif y_test[i]==1:
    if ypred[i]==-1:
      FN += 1
    elif ypred[i]==1:
      TP += 1
confusion_matrix = pd.DataFrame(data=[[TN,FP],[FN,TP]])
confusion_matrix

,0,1
0,19,2
1,2,19


In [44]:
SE = TP/(TP+FN)
SP = TN/(TN+FP)
Acc = (TP+TN)/(TP+TN+FP+FN)
print(SE)
print(SP)
print(Acc)

0.9047619047619048
0.9047619047619048
0.9047619047619048


**RBF**

In [49]:
X_train_size=int(X_train.size/19)

gamma_vals=np.linspace(0.1,10,100)
kernel_matrix_lst=[]
delta_lst=[]
acc_lst=[]
for gamma in gamma_vals:
 delta=np.zeros(X_train_size)
 kernel_matrix=np.zeros((X_size,X_size))
 for i in range(X_size):
  for j in range(X_size):
    kernel_matrix[i][j]=math.exp(-1*gamma* np.sum(np. power((X[i]-X[j]),2)))
 #training the model 
 for i in range(X_train_size):
  hypothesis=0
  
  for j in range(X_train_size):
    hypothesis=hypothesis+(delta[j]*y_train[j]*kernel_matrix[j][i])
  if(hypothesis>=0):
    h=1
  else:
    h=-1
  if(h!=y_train[i]):
    delta[i]=delta[i]+1 
 ##validation of data    
 X_val_size=int(X_val.size/19)
 count=0
 for i in range(X_val_size):
  hypothesis=0
  for j in range(X_train_size):
    hypothesis=hypothesis+(delta[j]*y_train[j]*kernel_matrix[j][i+X_train_size])
  if(hypothesis>=0):
    h=1
  else:
    h=-1
  if(h==y_val[i]):
    count=count+1
 accuracy=count/X_val_size
 acc_lst.append(accuracy)  
 delta_lst.append(delta)
 kernel_matrix_lst.append(kernel_matrix)


In [54]:
accuracy=0
index=0
for i in range(len(acc_lst)):
  if(acc_lst[i]>=accuracy):
    accuracy=acc_lst[i]
    index=i
acc_lst[index] 

0.918235294117647

In [60]:
##for testing data
X_test_size=int(X_test.size/19)
ypred=np.zeros(X_test_size)
for i in range(X_test_size):
  hypothesis=0
  for j in range(X_train_size):
    hypothesis=hypothesis+(delta_lst[index][j]*y_train[j]*(kernel_matrix_lst[index][j][i+X_train_size+X_val_size]))
  if(hypothesis>=0):
    ypred[i]=1
  else:
    ypred[i]=-1
TN = 0
TP = 0
FN = 0
FP = 0
for i in range(len(y_test)):
  if y_test[i]==-1:
    if ypred[i]==-1:
      TN += 1
    elif ypred[i]==1:
      FP += 1
  elif y_test[i]==1:
    if ypred[i]==-1:
      FN += 1
    elif ypred[i]==1:
      TP += 1

confusion_matrix = pd.DataFrame(data=[[TN,FP],[FN,TP]])
confusion_matrix


,0,1
0,17,4
1,2,19


In [61]:
SE = TP/(TP+FN)
SP = TN/(TN+FP)
Acc = (TP+TN)/(TP+TN+FP+FN)
print(SE)
print(SP)
print(Acc)

0.9047619047619048
0.8095238095238095
0.8571428571428571
